In [5]:
# Make sure libraries are reloaded when this cell is run
%load_ext autoreload
%autoreload 2

from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import OrbitPlotter3D
import copy
import CustomManeuvres
from poliastro.twobody.thrust.change_a_inc import change_a_inc
from poliastro._math.linalg import norm
import poliastro.core.angles as angles
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# Define the two coplanar debris
a = 3000+6371 << u.km
ecc = 0 << u.one
inc = 0 << u.deg
raan = 0 << u.deg
argp = 0 << u.deg
nu = 10 << u.deg

deb_1 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

a = 6000+6371 << u.km
inc = 45 << u.deg
nu = 60 << u.deg

deb_2 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

# Init the otv at debris 1
otv = copy.copy(deb_1)

In [109]:
print(deb_1.epoch.iso)
print(deb_2.epoch.iso)
print(otv.epoch.iso)

2000-01-01 12:00:00.000
2000-01-01 12:00:00.000
2000-01-01 12:00:00.000


In [20]:
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

In [21]:
import CustomManeuvres
inc_change = CustomManeuvres.simple_inc_change(otv, deb_2)

# Get the transfer time of the hoh_phas
transfer_time = inc_change.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Apply the maneuver to the otv
otv = otv.apply_maneuver(inc_change)

print('otv inc', otv.inc << u.deg)
print('deb_2 inc', deb_2.inc)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

currrent_anomaly 10.0 deg
thrust_location 179.999 deg
time_to_thrust 4263.180818936897 s
thrust_norm 4991.666653212488 m / s
y_thrust 1910.2281280737152 m / s
z_thrust -4611.698654022133 m / s
otv inc 44.999999998192656 deg
deb_2 inc 45.0 deg


In [22]:
print(deb_1.nu << u.deg)
print(otv.nu << u.deg)
print(deb_2.nu << u.deg)
print((otv.nu + otv.argp) << u.deg)

# mean_anomaly = angles.nu_to_M(otv.nu, otv.ecc)

# print(mean_anomaly << u.deg)

179.99899999999997 deg
89.99929289285737 deg
172.07740403789592 deg
180.0 deg


In [23]:
otv.argp = 0 << u.deg

In [24]:
print(deb_1.nu << u.deg)
print(otv.nu << u.deg)
print(deb_2.nu << u.deg)
print((otv.nu + otv.argp) << u.deg)

179.99899999999997 deg
89.99929289285737 deg
172.07740403789592 deg
89.99929289285737 deg


In [17]:
# Calculate a basic homnann
import CustomManeuvres

# Force argp
otv.argp = 0 << u.deg

hoh_phas = CustomManeuvres.hohmann_with_phasing(otv, deb_2)
hoh_phas

Target Delta:  31.724451800041155 deg
current_delta :  82.07811114503855 deg
dist:  50.353659344997396 deg
t_1:  3706.1609094411574 s
mean_anomaly_i:  -122.21456691526672 deg
new delta:  31.725533196594455 deg


Number of impulses: 2, Total cost: 0.841563 km / s

In [18]:
hoh_phas.impulses

((<Quantity 33931.51750328 s>,
  <Quantity [297.99784075,  74.61079575,   0.        ] m / s>),
 (<Quantity 5255.47453664 s>,
  <Quantity [-283.91309118,  -71.08434613,    0.        ] m / s>))

In [18]:
# Get the transfer time of the hoh_phas
transfer_time = hoh_phas.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Propagate the otv using the calculated thrust sequence
transfer_orbit, otv = otv.apply_maneuver(hoh_phas, intermediate=True)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)
frame.plot(transfer_orbit)



In [18]:
# Wrong values, need to use mean anomaly to compare
print("OTV ANGLE: ", otv.nu.to(u.deg))
print("DEB2 ANGLE: ", deb_2.nu.to(u.deg))
print('difference: ', (otv.nu - deb_2.nu).to(u.deg))
print('OTV.a: ', otv.a)
print('Deb2.a: ', deb_2.a)

OTV ANGLE:  -172.61536114641075 deg
DEB2 ANGLE:  157.19332482068768 deg
difference:  -329.8086859670984 deg
OTV.a:  9371.0 km
Deb2.a:  9371.0 km
